In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import numpy as np
import time
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.ops.gen_array_ops import shape
import random
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

## Model

In [41]:
def Def_model(X_test):
  input = tf.keras.Input(shape=X_test[0].shape)
  # CNN
  x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(input)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(3, padding='same')(x)
  x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.MaxPooling1D(3, padding='same')(x)
  x = tf.keras.layers.Flatten()(x)

  output = tf.keras.layers.Dense(64)(x)
  output = tf.keras.layers.Dense(2)(output)
  # output = tf.keras.layers.Dropout(.5,input_shape=(5,))(output)
  output = tf.keras.activations.sigmoid(output)
  global_model = tf.keras.Model(inputs=[input], outputs=[output])
  return global_model

# Hàm đánh giá

In [29]:
def Evaluation_Function(global_model, X_test, y_test):
    y_pred = global_model.predict(X_test).argmax(axis=1)
    y_true = np.array(y_test).argmax(axis=1)

    accuracy = tf.keras.metrics.Accuracy()
    accuracy.update_state(y_true, y_pred)
    acc = accuracy.result().numpy()

    recall = tf.keras.metrics.Recall()
    recall.update_state(y_true, y_pred)
    rec = recall.result().numpy()

    precision = tf.keras.metrics.Precision()
    precision.update_state(y_true, y_pred)
    prec = precision.result().numpy()
    f_score = 2 * (prec * rec) / (prec + rec)

    return [acc, rec, prec, f_score]

#Load func

In [30]:
def load_data(path, test = True,r = 1, agents = 1):
    if test:
        X = pd.read_csv(path)
    else:
        path = path + "SB19_CCFDUBL_BAL_P" + str(agents)+ "_2C" + ".csv"
        X = pd.read_csv(path)

    Y = X.iloc[:,-1:]
    num_records = Y.shape[0]
    X = tf.constant(X.values[:,0:30])
    X = tf.reshape(X, (num_records, 30, 1))
    Y = tf.one_hot(Y.values[:,0:], 2)
    Y = tf.reshape(Y, (num_records, 2))
    return X,Y

In [31]:
path = "/content/drive/MyDrive/SwarmLearn/Fraunt/Dataset/SB19_CCFDUBL_BAL_TEST_2C.csv"
X_test, Y_test = load_data(path)

In [42]:
from keras.optimizers import SGD
global_model = Def_model(X_test)
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.01)
global_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy"])

# Dữ liệu bình thường

In [43]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    list_agents_model = []
    avg_weights = []
    global_weights = global_model.get_weights()
    sum_data_train = 0
    for i in global_weights:
        avg_weights.append(np.zeros_like(i))
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(
            tf.keras.models.clone_model(global_model)
        )
        list_agents_model[i].set_weights(global_weights)
    model_save = global_model.get_weights()
    for i in range(agents):
      path = "/content/drive/MyDrive/SwarmLearn/Fraunt/Dataset/"
      X_train, Y_train = load_data(path,False, r, i)
      list_agents_model[i].compile(
          loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 32, epochs= 5,shuffle=False)

      local_weights = list_agents_model[i].get_weights()
      for j in range(len(local_weights)):
        avg_weights[j] += len(Y_train)* (local_weights[j] - model_save[j])
      sum_data_train += len(Y_train)
      for j in range(len(avg_weights)):
          avg_weights[j] = 0.6 * avg_weights[j] / sum_data_train
      global_model = tf.keras.models.clone_model(list_agents_model[0])
      global_model.compile(
              loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=["accuracy"]
          )
    for j in range(len(avg_weights)):
        avg_weights[j] = avg_weights[j] + model_save[j]
    global_model.set_weights(avg_weights)
  print('Final Evaluating:')
  acc,recall,precision,f_score = Evaluation_Function(global_model,X_test,Y_test)
  return acc,recall,precision,f_score

In [45]:
FL_Train(rounds=25,agents=3,global_model=global_model)

Round  1
Local 0 Training:
Epoch 1/5
8/8 [==============================] - 2s 6ms/step - loss: 0.6273 - accuracy: 0.6667
Epoch 2/5
8/8 [==============================] - 0s 5ms/step - loss: 0.4965 - accuracy: 0.7456
Epoch 3/5
8/8 [==============================] - 0s 5ms/step - loss: 0.4369 - accuracy: 0.7939
Epoch 4/5
8/8 [==============================] - 0s 5ms/step - loss: 0.3972 - accuracy: 0.8026
Epoch 5/5
8/8 [==============================] - 0s 5ms/step - loss: 0.3686 - accuracy: 0.8202
Local 1 Training:
Epoch 1/5
8/8 [==============================] - 1s 7ms/step - loss: 0.4680 - accuracy: 0.8070
Epoch 2/5
8/8 [==============================] - 0s 6ms/step - loss: 0.2439 - accuracy: 0.9386
Epoch 3/5
8/8 [==============================] - 0s 7ms/step - loss: 0.1420 - accuracy: 0.9781
Epoch 4/5
8/8 [==============================] - 0s 7ms/step - loss: 0.0980 - accuracy: 0.9912
Epoch 5/5
8/8 [==============================] - 0s 7ms/step - loss: 0.0738 - accuracy: 0.9912
Local

(0.89181286, 0.7865497, 0.9962963, 0.8790849258030239)

# Dữ liệu lật nhãn 10 20 30

In [ ]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    list_agents_model = []
    avg_weights = []
    global_weights = global_model.get_weights()
    sum_data_train = 0
    for i in global_weights:
        avg_weights.append(np.zeros_like(i))
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(
            tf.keras.models.clone_model(global_model)
        )
        list_agents_model[i].set_weights(global_weights)
    model_save = global_model.get_weights()
    for i in range(agents):
      path = "/content/drive/MyDrive/SwarmLearn/Fraunt/Flip_data/"
      X_train, Y_train = load_data(path,False, r, i)
      list_agents_model[i].compile(
          loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 32, epochs= 5,shuffle=False)

      local_weights = list_agents_model[i].get_weights()
      for j in range(len(local_weights)):
        avg_weights[j] += len(Y_train)* (local_weights[j] - model_save[j])
      sum_data_train += len(Y_train)
      for j in range(len(avg_weights)):
          avg_weights[j] = 0.6 * avg_weights[j] / sum_data_train
      global_model = tf.keras.models.clone_model(list_agents_model[0])
      global_model.compile(
              loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=["accuracy"]
          )
    for j in range(len(avg_weights)):
        avg_weights[j] = avg_weights[j] + model_save[j]
    global_model.set_weights(avg_weights)
  print('Final Evaluating:')
  acc,recall,precision,f_score = Evaluation_Function(global_model,X_test,Y_test)
  return acc,recall,precision,f_score

In [ ]:
FL_Train(rounds=5,agents=3,global_model=global_model)

Round  1
Local 0 Training:
Epoch 1/5
8/8 [==============================] - 1s 3ms/step - loss: 1.2542 - accuracy: 0.4781
Epoch 2/5
8/8 [==============================] - 0s 3ms/step - loss: 0.7355 - accuracy: 0.5965
Epoch 3/5
8/8 [==============================] - 0s 2ms/step - loss: 0.6053 - accuracy: 0.6667
Epoch 4/5
8/8 [==============================] - 0s 3ms/step - loss: 0.5481 - accuracy: 0.7061
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 0.5128 - accuracy: 0.7281
Local 1 Training:
Epoch 1/5
8/8 [==============================] - 0s 3ms/step - loss: 1.2542 - accuracy: 0.4781
Epoch 2/5
8/8 [==============================] - 0s 3ms/step - loss: 0.7355 - accuracy: 0.5965
Epoch 3/5
8/8 [==============================] - 0s 2ms/step - loss: 0.6053 - accuracy: 0.6667
Epoch 4/5
8/8 [==============================] - 0s 3ms/step - loss: 0.5481 - accuracy: 0.7061
Epoch 5/5
8/8 [==============================] - 0s 2ms/step - loss: 0.5128 - accuracy: 0.7281
Local

(0.73245615, 0.60818714, 0.8093385, 0.6944908308060159)

# Dữ liệu lật nhãn 0 100 30

In [ ]:
def FL_Train(rounds,agents,global_model):
  for r in range(rounds):
    list_agents_model = []
    avg_weights = []
    global_weights = global_model.get_weights()
    sum_data_train = 0
    for i in global_weights:
        avg_weights.append(np.zeros_like(i))
    print("Round ", r + 1)
    # Spread Model
    for i in range(agents):
        list_agents_model.append(
            tf.keras.models.clone_model(global_model)
        )
        list_agents_model[i].set_weights(global_weights)
    model_save = global_model.get_weights()
    for i in range(agents):
      path = "/content/drive/MyDrive/SwarmLearn/Fraunt/Flip_data2/"
      X_train, Y_train = load_data(path,False, r, i)
      list_agents_model[i].compile(
          loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=["accuracy"]
      )
      print('Local '+str(i)+' Training:')
      list_agents_model[i].fit(X_train, Y_train, batch_size = 32, epochs= 5,shuffle=False)

      local_weights = list_agents_model[i].get_weights()
      for j in range(len(local_weights)):
        avg_weights[j] += len(Y_train)* (local_weights[j] - model_save[j])
      sum_data_train += len(Y_train)
      for j in range(len(avg_weights)):
          avg_weights[j] = 0.6 * avg_weights[j] / sum_data_train
      global_model = tf.keras.models.clone_model(list_agents_model[0])
      global_model.compile(
              loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=["accuracy"]
          )
    for j in range(len(avg_weights)):
        avg_weights[j] = avg_weights[j] + model_save[j]
    global_model.set_weights(avg_weights)
  print('Final Evaluating:')
  acc,recall,precision,f_score = Evaluation_Function(global_model,X_test,Y_test)
  return acc,recall,precision,f_score

In [ ]:
FL_Train(rounds=5,agents=3,global_model=global_model)

Round  1
Local 0 Training:
Epoch 1/5
8/8 [==============================] - 1s 4ms/step - loss: 1.1369 - accuracy: 0.5658
Epoch 2/5
8/8 [==============================] - 0s 4ms/step - loss: 0.8633 - accuracy: 0.6316
Epoch 3/5
8/8 [==============================] - 0s 3ms/step - loss: 0.7917 - accuracy: 0.6535
Epoch 4/5
8/8 [==============================] - 0s 3ms/step - loss: 0.7406 - accuracy: 0.6579
Epoch 5/5
8/8 [==============================] - 0s 3ms/step - loss: 0.6992 - accuracy: 0.6754
Local 1 Training:
Epoch 1/5
8/8 [==============================] - 1s 3ms/step - loss: 0.9212 - accuracy: 0.6316
Epoch 2/5
8/8 [==============================] - 0s 3ms/step - loss: 0.4112 - accuracy: 0.8114
Epoch 3/5
8/8 [==============================] - 0s 3ms/step - loss: 0.2954 - accuracy: 0.8947
Epoch 4/5
8/8 [==============================] - 0s 3ms/step - loss: 0.2298 - accuracy: 0.9211
Epoch 5/5
8/8 [==============================] - 0s 3ms/step - loss: 0.1864 - accuracy: 0.9386
Local

(0.6388889, 0.6725146, 0.63013697, 0.6506364623204313)